**Notebook to compare Akantu and WOBBLE for accelerating rotation about one axis**

In [ ]:
import sys
sys.path.append('../..')

import numpy as np
import akantu as aka
from tqdm import tqdm
from wobble.rb_simple import SimpleRB
from wobble.rb_coupled import CoupledRB
from wobble.pure import PureMA
import matplotlib.pyplot as plt

In [ ]:
# SimpleRB Simulation
sim=SimpleRB('Simple_RB', 3, 0.01, 1,
                 mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                 num_modes=10,
                 force_path='../sample_data_files/force_files/pure_rotation_100.txt',
                 eigenmode_path='../sample_data_files/eigenmode_files/rb_modes.csv')
sim.solve()

In [ ]:
# CoupledRB Simulation
cup=CoupledRB('Double_RB', 3, 0.01, 1,
                 mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                 num_modes=10,
                 force_path='../sample_data_files/force_files/pure_rotation_100.txt',
                 eigenmode_path='../sample_data_files/eigenmode_files/rb_modes.csv')
cup.solve()

In [ ]:
# Running the Akantu simulation
spatial_dimension = 3
mesh_file = '../sample_data_files/mesh_files/beam.msh'

aka.parseInput('../sample_data_files/material_files/steel.dat')
mesh = aka.Mesh(spatial_dimension)
mesh.read(mesh_file)

model = aka.SolidMechanicsModel(mesh)

model.initFull(_analysis_method=aka._explicit_lumped_mass)

displacement = model.getDisplacement()
force = model.getExternalForce()

for row in range(force.shape[0]):
    force[row] = sim.unprojected_force[:, 0].reshape(-1, 3)[row]

time_factor = 0.8
stable_time_step = model.getStableTimeStep() * time_factor
model.setTimeStep(stable_time_step)
max_steps = int(1 / stable_time_step)


force_times = np.hstack((sim.force_times, 1e5))

disps=[]
j = 1
for i in tqdm(range(0, max_steps + 1)):
    
    if i * stable_time_step >= force_times[j]:
        for row in range(force.shape[0]):
            force[row] = sim.unprojected_force[:, j].reshape(-1, 3)[row]
            
        j += 1
    
    model.solveStep()
    if i % 1000 == 0:
        disp=model.getDisplacement()
        disps.append(disp.copy())

disps = np.array(disps)

In [ ]:
# Plotting parameters
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

In [ ]:
fig, axs = plt.subplots(1, 3)
axes = ['x', 'y', 'z']
for i in range(3):
    axs[i].plot(sim.time_array, sim.total_displacement_vectors[i] * 100, label='SimpleRB')
    axs[i].plot(sim.time_array, cup.total_displacement_vectors[i] * 100, label='CoupledRB')
    axs[i].plot(np.linspace(0,1,116), disps[:, 0, i] * 100, label='Akantu')
    axs[i].legend()
    axs[i].set_title(f'Displacement in {axes[i]}', fontweight='bold')

    if i == 0:
        axs[i].set_ylabel('Displacement [cm]')
        
    if i == 1:
        axs[i].set_xlabel('Time [s]')
    
fig.suptitle('Accelerating rotation about the x axis', fontweight='bold')
fig.set_size_inches(12.5, 5.5)
fig.tight_layout()
fig.show()

In [ ]:
# Run SimpleRB for an initial angular acceleration of rb_omega = [1, 0, 0]^T
sim2=SimpleRB('SimpleRB_2', 3, 0.01, 1,
                 mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                 num_modes=10,
                 force_path='../sample_data_files/force_files/no_force.txt',
                 eigenmode_path='../sample_data_files/eigenmode_files/rb_modes.csv')
sim2.solve(rb_omega_0=[1, 0, 0])

In [ ]:
# Design the initial velocity condition for Akantu. It is given by the cross product of rb_omega x b
b = (sim2.mesh_nodes - sim2.cm)
v_0 = np.cross(np.array([1, 0, 0]), b)

In [ ]:
# Set the initial conditions in Akantu and run the simulation

velocity = model.getVelocity()

for row in range(v_0.shape[0]):
    velocity[row] = v_0[row]

for row in range(force.shape[0]):
    force[row] = [0,0,0]
    
for row in range(displacement.shape[0]):
    displacement[row] = [0,0,0]
    
disps2=[]
for i in tqdm(range(0, max_steps + 1)):
    
    if i % 1000 == 0:
        print(velocity.sum(axis=0))
        print(force.sum())
        print(displacement.sum())
    
    model.solveStep()
    if i % 1000 == 0:
        disp=model.getDisplacement()
        disps2.append(disp.copy())

disps2 = np.array(disps2)

In [ ]:
# Format the plot and plot the results
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"


fig, axs = plt.subplots(1, 3)
axes = ['x', 'y', 'z']
for i in range(3):
    axs[i].plot(sim2.time_array, sim2.total_displacement_vectors[i] * 100, label='SimpleRB')
    axs[i].plot(np.linspace(0,1,116), disps2[:, 0, i] * 100, label='Akantu')
    axs[i].legend()
    axs[i].set_title(f'Displacement in {axes[i]}', fontweight='bold')

    if i == 0:
        axs[i].set_ylabel('Displacement [cm]')
        
    if i == 1:
        axs[i].set_xlabel('Time [s]')
    
fig.suptitle('Constant rotation about the x axis', fontweight='bold')
fig.set_size_inches(12.5, 5.5)
fig.tight_layout()
fig.show()